In [1]:
import os, sys
sys.path.insert(0, os.path.abspath(".."))

In [2]:
import boto3
import pickle
import numpy as np
import pandas as pd
import datetime
from datetime import datetime as dt
from dateutil.relativedelta import *

In [3]:
pd.options.display.max_columns = None

In [4]:
session = boto3.session.Session()
s3client = session.client('s3')

In [5]:
response = s3client.get_object(Bucket='inai-summerofdata', Key='mlpreproc/pickles_temporalidad/df_preproc.pkl')

body = response['Body'].read()
dum_df = pickle.loads(body)

In [6]:
response = s3client.get_object(Bucket='inai-summerofdata', Key='mlpreproc/pickles_temporalidad/index_cv_split.pkl')

body = response['Body'].read()
index_output = pickle.loads(body)

In [7]:
dum_df.head()

,folio,fechasolicitud,anosolicitud,messolicitud,semanasolicitud,diasolicitud,descripcionsolicitud_lemma,solicitud_lemma_longitud,dependencia_clean_administracion portuaria integral,dependencia_clean_administracion publica federal,dependencia_clean_aeropuertos y servicios auxiliares (asa),dependencia_clean_archivo general de la nación,dependencia_clean_auditoría superior de la federación,dependencia_clean_banca de desarrollo,dependencia_clean_camara de dipuados,dependencia_clean_capufe,dependencia_clean_cfe,dependencia_clean_cnbv,dependencia_clean_cofepris,dependencia_clean_conacyt,dependencia_clean_empresas de participacion estatal,dependencia_clean_hospitales,dependencia_clean_inah,dependencia_clean_inai,dependencia_clean_ine,dependencia_clean_instituciones de educacion superior autonomas,dependencia_clean_instituto del fondo nacional de la vivienda para los trabajadores,dependencia_clean_instituto federal de telecomunicaciones (ift),dependencia_clean_instituto mexicano de la propiedad industrial,dependencia_clean_instituto mexicano del seguro social (imss),dependencia_clean_instituto nacional de migración,dependencia_clean_ipn,dependencia_clean_lotenal,dependencia_clean_organismo autonomo,dependencia_clean_organismo descentralizado,dependencia_clean_partidos politicos,dependencia_clean_pemex,dependencia_clean_pgr,dependencia_clean_poder judicial de la federacion,dependencia_clean_policia federal,dependencia_clean_presidencia de la republica,dependencia_clean_procuraduría federal del consumidor,dependencia_clean_profeco,dependencia_clean_registro agrario nacional,dependencia_clean_sader,dependencia_clean_sae,dependencia_clean_sagarpa,dependencia_clean_sat,dependencia_clean_scjn,dependencia_clean_sct,dependencia_clean_se,dependencia_clean_secretaria de bienestar,dependencia_clean_secretariado ejecutivo del sistema nacional de seguridad pública,dependencia_clean_secretaría de cultura,dependencia_clean_sectur,dependencia_clean_sedatu,dependencia_clean_sedena,dependencia_clean_sedesol,dependencia_clean_segob,dependencia_clean_semar,dependencia_clean_semarnat,dependencia_clean_senado de la república,dependencia_clean_sener,dependencia_clean_sep,dependencia_clean_sfp,dependencia_clean_shcp,dependencia_clean_sindicatos,dependencia_clean_sre,dependencia_clean_ssa,dependencia_clean_sspc,dependencia_clean_stps,dependencia_clean_tribunales administrativos,dependencia_clean_unam,codigo_calidad_respuesta_real
0,1857500001912,2012-01-01 05:04:07,2012,1,52,6,comportamiento historico tipicos comprender es...,424,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1857500002012,2012-01-01 05:13:16,2012,1,52,6,usar flujo efectivo pozo tipico aguar fria ela...,289,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0001800000712,2012-01-01 09:46:39,2012,1,52,6,proporcionar magnetico relacion ocupar amparar...,143,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,0000600003712,2012-01-01 10:11:41,2012,1,52,6,proporcionar magnetico relacion ocupar amparar...,143,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
4,0001500002812,2012-01-01 10:32:15,2012,1,52,6,proporcionar magnetico relacion ocupar amparar...,188,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Modeling

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from pprint import pprint

In [9]:
drop_columns = ['folio', 'codigo_calidad_respuesta_real']

In [10]:
X = dum_df.drop(drop_columns, axis=1)
y = pd.DataFrame(dum_df['codigo_calidad_respuesta_real'])

In [11]:
X.head()

,fechasolicitud,anosolicitud,messolicitud,semanasolicitud,diasolicitud,descripcionsolicitud_lemma,solicitud_lemma_longitud,dependencia_clean_administracion portuaria integral,dependencia_clean_administracion publica federal,dependencia_clean_aeropuertos y servicios auxiliares (asa),dependencia_clean_archivo general de la nación,dependencia_clean_auditoría superior de la federación,dependencia_clean_banca de desarrollo,dependencia_clean_camara de dipuados,dependencia_clean_capufe,dependencia_clean_cfe,dependencia_clean_cnbv,dependencia_clean_cofepris,dependencia_clean_conacyt,dependencia_clean_empresas de participacion estatal,dependencia_clean_hospitales,dependencia_clean_inah,dependencia_clean_inai,dependencia_clean_ine,dependencia_clean_instituciones de educacion superior autonomas,dependencia_clean_instituto del fondo nacional de la vivienda para los trabajadores,dependencia_clean_instituto federal de telecomunicaciones (ift),dependencia_clean_instituto mexicano de la propiedad industrial,dependencia_clean_instituto mexicano del seguro social (imss),dependencia_clean_instituto nacional de migración,dependencia_clean_ipn,dependencia_clean_lotenal,dependencia_clean_organismo autonomo,dependencia_clean_organismo descentralizado,dependencia_clean_partidos politicos,dependencia_clean_pemex,dependencia_clean_pgr,dependencia_clean_poder judicial de la federacion,dependencia_clean_policia federal,dependencia_clean_presidencia de la republica,dependencia_clean_procuraduría federal del consumidor,dependencia_clean_profeco,dependencia_clean_registro agrario nacional,dependencia_clean_sader,dependencia_clean_sae,dependencia_clean_sagarpa,dependencia_clean_sat,dependencia_clean_scjn,dependencia_clean_sct,dependencia_clean_se,dependencia_clean_secretaria de bienestar,dependencia_clean_secretariado ejecutivo del sistema nacional de seguridad pública,dependencia_clean_secretaría de cultura,dependencia_clean_sectur,dependencia_clean_sedatu,dependencia_clean_sedena,dependencia_clean_sedesol,dependencia_clean_segob,dependencia_clean_semar,dependencia_clean_semarnat,dependencia_clean_senado de la república,dependencia_clean_sener,dependencia_clean_sep,dependencia_clean_sfp,dependencia_clean_shcp,dependencia_clean_sindicatos,dependencia_clean_sre,dependencia_clean_ssa,dependencia_clean_sspc,dependencia_clean_stps,dependencia_clean_tribunales administrativos,dependencia_clean_unam
0,2012-01-01 05:04:07,2012,1,52,6,comportamiento historico tipicos comprender es...,424,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2012-01-01 05:13:16,2012,1,52,6,usar flujo efectivo pozo tipico aguar fria ela...,289,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2012-01-01 09:46:39,2012,1,52,6,proporcionar magnetico relacion ocupar amparar...,143,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,2012-01-01 10:11:41,2012,1,52,6,proporcionar magnetico relacion ocupar amparar...,143,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,2012-01-01 10:32:15,2012,1,52,6,proporcionar magnetico relacion ocupar amparar...,188,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
y.head()

,codigo_calidad_respuesta_real
0,1
1,1
2,0
3,1
4,0


In [13]:
features_cv = np.array(X.drop('fechasolicitud', axis=1))
labels_cv = np.array(y)
labels_cv = np.ravel(labels_cv)

In [28]:
ngram_range = (1, 2)
min_df = 10
max_df = 1.
max_features = 300

In [29]:
preprocess = ColumnTransformer(
    [('descripcionsolicitud_tfidf', TfidfVectorizer(encoding='utf-8',
                                        ngram_range=ngram_range,
                                        stop_words=None,
                                        lowercase=False,
                                        max_df=max_df,
                                        min_df=min_df,
                                        max_features=max_features,
                                        norm='l2',
                                        sublinear_tf=True), 4)],
    remainder='passthrough')

In [31]:
clf = RandomForestClassifier(random_state=8)
clf.get_params()

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 'warn',
 'n_jobs': None,
 'oob_score': False,
 'random_state': 8,
 'verbose': 0,
 'warm_start': False}

In [17]:
rfc = Pipeline([('preprocess', preprocess), ('clf', clf)])

In [32]:
n_estimators = [int(x) for x in np.linspace(start=200, stop=1000, num=5)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(20, 100, num=5)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {'clf__n_estimators': n_estimators,
              'clf__max_features': max_features,
              'clf__max_depth': max_depth,
              'clf__min_samples_split': min_samples_split,
              'clf__min_samples_leaf': min_samples_leaf,
              'clf__bootstrap': bootstrap}

pprint(random_grid)

{'clf__bootstrap': [True, False],
 'clf__max_depth': [20, 40, 60, 80, 100, None],
 'clf__max_features': ['auto', 'sqrt'],
 'clf__min_samples_leaf': [1, 2, 4],
 'clf__min_samples_split': [2, 5, 10],
 'clf__n_estimators': [200, 400, 600, 800, 1000]}


In [33]:
random_search = RandomizedSearchCV(estimator=rfc,
                                  param_distributions=random_grid,
                                  n_jobs=-1,
                                  n_iter=10,
                                  scoring='accuracy',
                                  cv=index_output,
                                  verbose=1,
                                  random_state=8)

random_search.fit(features_cv, labels_cv)

Fitting 204 folds for each of 10 candidates, totalling 2040 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  7.7min
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed: 83.5min
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed: 196.7min
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed: 264.8min
[Parallel(n_jobs=-1)]: Done 1226 tasks      | elapsed: 361.7min
[Parallel(n_jobs=-1)]: Done 1776 tasks      | elapsed: 515.2min
[Parallel(n_jobs=-1)]: Done 2040 out of 2040 | elapsed: 691.2min finished
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the

RandomizedSearchCV(cv=[([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,
                         16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, ...],
                        [13889, 13890, 13891, 13892, 13893, 13894, 13895, 13896,
                         13897, 13898, 13899, 13900, 13901, 13902, 13903, 13904,
                         13905, 13906, 13907, 13908, 13909, 13910, 13911, 13912,
                         13913, 13914, 13915, 13916, 13917, 13918, ...]),
                       ([2794, 2795, 2796, 2797, 2798, 2799, 2800, 2801, 2802,
                         2803, 2...
                   iid='warn', n_iter=10, n_jobs=-1,
                   param_distributions={'clf__bootstrap': [True, False],
                                        'clf__max_depth': [20, 40, 60, 80, 100,
                                                           None],
                                        'clf__max_features': ['auto', 'sqrt'],
                                        'clf__min_s

In [34]:
print(random_search.best_params_)

{'clf__n_estimators': 400, 'clf__min_samples_split': 10, 'clf__min_samples_leaf': 4, 'clf__max_features': 'sqrt', 'clf__max_depth': 100, 'clf__bootstrap': False}


In [35]:
print(random_search.best_score_)

0.6537616411465165


In [36]:
pd.DataFrame(random_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__n_estimators,param_clf__min_samples_split,param_clf__min_samples_leaf,param_clf__max_features,param_clf__max_depth,param_clf__bootstrap,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,split10_test_score,split11_test_score,split12_test_score,split13_test_score,split14_test_score,split15_test_score,split16_test_score,split17_test_score,split18_test_score,split19_test_score,split20_test_score,split21_test_score,split22_test_score,split23_test_score,split24_test_score,split25_test_score,split26_test_score,split27_test_score,split28_test_score,split29_test_score,split30_test_score,split31_test_score,split32_test_score,split33_test_score,split34_test_score,split35_test_score,split36_test_score,split37_test_score,split38_test_score,split39_test_score,split40_test_score,split41_test_score,split42_test_score,split43_test_score,split44_test_score,split45_test_score,split46_test_score,split47_test_score,split48_test_score,split49_test_score,split50_test_score,split51_test_score,split52_test_score,split53_test_score,split54_test_score,split55_test_score,split56_test_score,split57_test_score,split58_test_score,split59_test_score,split60_test_score,split61_test_score,split62_test_score,split63_test_score,split64_test_score,split65_test_score,split66_test_score,split67_test_score,split68_test_score,split69_test_score,split70_test_score,split71_test_score,split72_test_score,split73_test_score,split74_test_score,split75_test_score,split76_test_score,split77_test_score,split78_test_score,split79_test_score,split80_test_score,split81_test_score,split82_test_score,split83_test_score,split84_test_score,split85_test_score,split86_test_score,split87_test_score,split88_test_score,split89_test_score,split90_test_score,split91_test_score,split92_test_score,split93_test_score,split94_test_score,split95_test_score,split96_test_score,split97_test_score,split98_test_score,split99_test_score,split100_test_score,split101_test_score,split102_test_score,split103_test_score,split104_test_score,split105_test_score,split106_test_score,split107_test_score,split108_test_score,split109_test_score,split110_test_score,split111_test_score,split112_test_score,split113_test_score,split114_test_score,split115_test_score,split116_test_score,split117_test_score,split118_test_score,split119_test_score,split120_test_score,split121_test_score,split122_test_score,split123_test_score,split124_test_score,split125_test_score,split126_test_score,split127_test_score,split128_test_score,split129_test_score,split130_test_score,split131_test_score,split132_test_score,split133_test_score,split134_test_score,split135_test_score,split136_test_score,split137_test_score,split138_test_score,split139_test_score,split140_test_score,split141_test_score,split142_test_score,split143_test_score,split144_test_score,split145_test_score,split146_test_score,split147_test_score,split148_test_score,split149_test_score,split150_test_score,split151_test_score,split152_test_score,split153_test_score,split154_test_score,split155_test_score,split156_test_score,split157_test_score,split158_test_score,split159_test_score,split160_test_score,split161_test_score,split162_test_score,split163_test_score,split164_test_score,split165_test_score,split166_test_score,split167_test_score,split168_test_score,split169_test_score,split170_test_score,split171_test_score,split172_test_score,split173_test_score,split174_test_score,split175_test_score,split176_test_score,split177_test_score,split178_test_score,split179_test_score,split180_test_score,split181_test_score,split182_test_score,split183_test_score,split184_test_score,split185_test_score,split186_test_score,split187_test_score,split188_test_score,split189_test_score,split190_test_score,split191_test_score,split192_test_score,split193_test_score,split194_test_sco

Guardamos en S3

In [ ]:
bucket='inai-summerofdata'
key='modeling/random_forest/1_temporalidad/random_search.pkl'
pickle_byte_obj = pickle.dumps(random_search)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, key).put(Body=pickle_byte_obj)

In [38]:
bootstrap = [False]
max_depth = [90, 100, 110]
max_features = ['sqrt']
min_samples_leaf = [2, 4, 6]
min_samples_split = [5, 10, 15]
n_estimators = [400]

param_grid = {
    'clf__bootstrap': bootstrap,
    'clf__max_depth': max_depth,
    'clf__max_features': max_features,
    'clf__min_samples_leaf': min_samples_leaf,
    'clf__min_samples_split': min_samples_split,
    'clf__n_estimators': n_estimators
}

In [39]:
grid_search = GridSearchCV(estimator=rfc,
                          param_grid=param_grid,
                          scoring='accuracy',
                          cv=index_output,
                          n_jobs=-1,
                          verbose=1)

grid_search.fit(features_cv , labels_cv)

Fitting 204 folds for each of 27 candidates, totalling 5508 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed: 49.3min
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed: 128.4min
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed: 222.9min
[Parallel(n_jobs=-1)]: Done 1226 tasks      | elapsed: 343.2min
[Parallel(n_jobs=-1)]: Done 1776 tasks      | elapsed: 445.0min
[Parallel(n_jobs=-1)]: Done 2426 tasks      | elapsed: 658.6min
[Parallel(n_jobs=-1)]: Done 3176 tasks      | elapsed: 839.7min
[Parallel(n_jobs=-1)]: Done 4026 tasks      | elapsed: 1037.0min
[Parallel(n_jobs=-1)]: Done 4976 tasks      | elapsed: 1256.4min
[Parallel(n_jobs=-1)]: Done 5508 out of 5508 | elapsed: 1359.7min finished
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in versi

GridSearchCV(cv=[([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,
                   18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, ...],
                  [13889, 13890, 13891, 13892, 13893, 13894, 13895, 13896,
                   13897, 13898, 13899, 13900, 13901, 13902, 13903, 13904,
                   13905, 13906, 13907, 13908, 13909, 13910, 13911, 13912,
                   13913, 13914, 13915, 13916, 13917, 13918, ...]),
                 ([2794, 2795, 2796, 2797, 2798, 2799, 2800, 2801, 2802, 2803,
                   2804, 28...
                                                               random_state=8,
                                                               verbose=0,
                                                               warm_start=False))],
                                verbose=False),
             iid='warn', n_jobs=-1,
             param_grid={'clf__bootstrap': [False],
                         'clf__max_depth': [90, 100, 110],
           

In [40]:
print(grid_search.best_params_)

{'clf__bootstrap': False, 'clf__max_depth': 110, 'clf__max_features': 'sqrt', 'clf__min_samples_leaf': 6, 'clf__min_samples_split': 5, 'clf__n_estimators': 400}


In [41]:
print(grid_search.best_score_)

0.6543296101780051


In [42]:
pd.DataFrame(grid_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__bootstrap,param_clf__max_depth,param_clf__max_features,param_clf__min_samples_leaf,param_clf__min_samples_split,param_clf__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,split10_test_score,split11_test_score,split12_test_score,split13_test_score,split14_test_score,split15_test_score,split16_test_score,split17_test_score,split18_test_score,split19_test_score,split20_test_score,split21_test_score,split22_test_score,split23_test_score,split24_test_score,split25_test_score,split26_test_score,split27_test_score,split28_test_score,split29_test_score,split30_test_score,split31_test_score,split32_test_score,split33_test_score,split34_test_score,split35_test_score,split36_test_score,split37_test_score,split38_test_score,split39_test_score,split40_test_score,split41_test_score,split42_test_score,split43_test_score,split44_test_score,split45_test_score,split46_test_score,split47_test_score,split48_test_score,split49_test_score,split50_test_score,split51_test_score,split52_test_score,split53_test_score,split54_test_score,split55_test_score,split56_test_score,split57_test_score,split58_test_score,split59_test_score,split60_test_score,split61_test_score,split62_test_score,split63_test_score,split64_test_score,split65_test_score,split66_test_score,split67_test_score,split68_test_score,split69_test_score,split70_test_score,split71_test_score,split72_test_score,split73_test_score,split74_test_score,split75_test_score,split76_test_score,split77_test_score,split78_test_score,split79_test_score,split80_test_score,split81_test_score,split82_test_score,split83_test_score,split84_test_score,split85_test_score,split86_test_score,split87_test_score,split88_test_score,split89_test_score,split90_test_score,split91_test_score,split92_test_score,split93_test_score,split94_test_score,split95_test_score,split96_test_score,split97_test_score,split98_test_score,split99_test_score,split100_test_score,split101_test_score,split102_test_score,split103_test_score,split104_test_score,split105_test_score,split106_test_score,split107_test_score,split108_test_score,split109_test_score,split110_test_score,split111_test_score,split112_test_score,split113_test_score,split114_test_score,split115_test_score,split116_test_score,split117_test_score,split118_test_score,split119_test_score,split120_test_score,split121_test_score,split122_test_score,split123_test_score,split124_test_score,split125_test_score,split126_test_score,split127_test_score,split128_test_score,split129_test_score,split130_test_score,split131_test_score,split132_test_score,split133_test_score,split134_test_score,split135_test_score,split136_test_score,split137_test_score,split138_test_score,split139_test_score,split140_test_score,split141_test_score,split142_test_score,split143_test_score,split144_test_score,split145_test_score,split146_test_score,split147_test_score,split148_test_score,split149_test_score,split150_test_score,split151_test_score,split152_test_score,split153_test_score,split154_test_score,split155_test_score,split156_test_score,split157_test_score,split158_test_score,split159_test_score,split160_test_score,split161_test_score,split162_test_score,split163_test_score,split164_test_score,split165_test_score,split166_test_score,split167_test_score,split168_test_score,split169_test_score,split170_test_score,split171_test_score,split172_test_score,split173_test_score,split174_test_score,split175_test_score,split176_test_score,split177_test_score,split178_test_score,split179_test_score,split180_test_score,split181_test_score,split182_test_score,split183_test_score,split184_test_score,split185_test_score,split186_test_score,split187_test_score,split188_test_score,split189_test_score,split190_test_score,split191_test_score,split192_test_score,split193_test_score,split194_test_sco

Guardamos en S3

In [ ]:
bucket='inai-summerofdata'
key='modeling/random_forest/1_temporalidad/grid_search.pkl'
pickle_byte_obj = pickle.dumps(grid_search)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, key).put(Body=pickle_byte_obj)

**Mejor modelo**

In [44]:
best_gbc = grid_search.best_estimator_

In [45]:
best_gbc

Pipeline(memory=None,
         steps=[('preprocess',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('descripcionsolicitud_tfidf',
                                                  TfidfVectorizer(analyzer='word',
                                                                  binary=False,
                                                                  decode_error='strict',
                                                                  dtype=<class 'numpy.float64'>,
                                                                  encoding='utf-8',
                                                                  input='content',
                                                                  lowercase=False,
                                                                  max_df=1.0,

In [ ]:
bucket='inai-summerofdata'
key='modeling/random_forest/1_temporalidad/best_gbc.pkl'
pickle_byte_obj = pickle.dumps(best_gbc)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, key).put(Body=pickle_byte_obj)

##  Fit y desempeño del modelo

In [47]:
training_accuracy = []
test_accuracy = []
for train_index, test_index in index_output:
    
    features_train = np.array(X.loc[train_index].drop('fechasolicitud', axis=1))
    labels_train = np.ravel(y.loc[train_index])
    
    features_test = np.array(X.loc[test_index].drop('fechasolicitud', axis=1))
    labels_test = np.ravel(y.loc[test_index])
    
    # Modeling train data
    best_gbc.fit(features_train, labels_train)
    
    # Predictions
    gbc_pred = best_gbc.predict(features_test)
    
    # Training accuracy
    training_accuracy.append(accuracy_score(labels_train, best_gbc.predict(features_train)))
    
    # Test accuracy
    test_accuracy.append(accuracy_score(labels_test, gbc_pred))
    
    

In [48]:
average_training_accuracy = np.mean(training_accuracy)
average_test_accuracy = np.mean(test_accuracy)

**Average training accuracy**

In [49]:
average_training_accuracy

0.8270634272961886

**Average test accuracy**

In [50]:
average_test_accuracy

0.653751613292889

Creamos un dataframe

In [51]:
data = {
    'Modelo': 'RFC',
    'Average Training Set Accuracy': average_training_accuracy,
    'Average Test Set Accuracy': average_test_accuracy
}
df_models_gbc = pd.DataFrame(data, index=[0])

In [52]:
df_models_gbc

,Modelo,Average Training Set Accuracy,Average Test Set Accuracy
0,RFC,0.827063,0.653752


Guardamos en S3

In [53]:
features = np.array(X.drop('fechasolicitud', axis=1))
labels = np.ravel(y)

In [ ]:
bucket='inai-summerofdata'
key='modeling/random_forest/1_temporalidad/features.pkl'
pickle_byte_obj = pickle.dumps(features)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, key).put(Body=pickle_byte_obj)

In [ ]:
bucket='inai-summerofdata'
key='modeling/random_forest/1_temporalidad/labels.pkl'
pickle_byte_obj = pickle.dumps(labels)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, key).put(Body=pickle_byte_obj)

In [ ]:
bucket='inai-summerofdata'
key='modeling/random_forest/1_temporalidad/df_models_gbc.pkl'
pickle_byte_obj = pickle.dumps(df_models_gbc)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, key).put(Body=pickle_byte_obj)